# Impervious Surfaces

A script to calculate the % coverage of impervious surface within each CSO Small Area.

## Script Setup

In [1]:
#load the libraries
library(sf)
library(rgdal)
library(rgeos)
library(raster)
library(tidyverse)

#install these packages if they are not installed
# install.packages('geojsonio')
# install.packages('exactextractr')

library(geojsonio)
library(exactextractr)

Linking to GEOS 3.12.1, GDAL 3.8.3, PROJ 9.3.1; sf_use_s2() is TRUE

Loading required package: sp

Please note that rgdal will be retired during October 2023,
plan transition to sf/stars/terra functions using GDAL and PROJ
at your earliest convenience.
See https://r-spatial.org/r/2023/05/15/evolution4.html and https://github.com/r-spatial/evolution
rgdal: version: 1.6-7, (SVN revision 1203)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.8.3, released 2024/01/04
Path to GDAL shared files: /Users/declan/anaconda3/envs/ISVEHI/share/gdal
 GDAL does not use iconv for recoding strings.
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 9.3.1, December 1st, 2023, [PJ_VERSION: 931]
Path to PROJ shared files: /Users/declan/Library/Application Support/proj:/Users/declan/anaconda3/envs/ISVEHI/share/proj
PROJ CDN enabled: TRUE
Linking to sp version:2.1-2
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rg

In [2]:
#set a directory for export 
exportDirectory <- "../1_InputData/1c_ImperviousSurface"

# Data

## Small Areas

In [3]:
#get the small area spatial data -Small_Areas_Ungeneralised__OSi_National_Statistical_Boundaries__2022
smallAreaShp <- st_read("./downloads/SMALL_AREA_2022.shp")
smallAreaShp <- smallAreaShp[c('SA_GUID__1', 'COUNTY_ENG')]


Reading layer `SMALL_AREA_2022' from data source 
  `/Users/declan/Projects/REACHOUT/walther/ISVEHI-main_2024/0_Scripts/downloads/SMALL_AREA_2022.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 18919 features and 28 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 417471.5 ymin: 519663.7 xmax: 734481.1 ymax: 966896.3
Projected CRS: IRENET95 / Irish Transverse Mercator


## Impervious Surface

The impervious surface data can be found at https://land.copernicus.eu/pan-european/high-resolution-layers/imperviousness/status-maps. Data from 2022 hjas been used in the ISVEHI. You will need to sign up/sign in to access this data.

Unzip the files, and place all of the extracted files into a directory. These two tiles need to be joined into one larger file.


In [4]:
# the directory of the impervious surface TIF tiles.
imperviousDir <- '/Users/declan/Projects/REACHOUT/walther/ISVEHI-main_2024/0_Scripts/downloads/Impervious/IMD_2018_010m_ie_03035_v020/DATA'

In [5]:
# merge the tiles into one (make take some time to complete)
files <- list.files(path = imperviousDir, pattern = ".tif$", full.names = TRUE)
raster <- lapply(files, raster)

imperviousRaster <- do.call(merge, c(raster, tolerance = 1))
# plot(imperviousRaster)
# read locally
# imperviousRaster <- raster('D:/Spatial_Data/Copernicus/LMS/IMD2015/Extracted/Processed/IMD_2015_20m_eu_03035_d05_IRL_WGS84.tif')

In [6]:
#convert to a simple feature
smallAreasSF <- st_as_sf(smallAreaShp)

#calculate the area of the raster cell (m2)
cellArea <- 10*10

#calculate the number of cells within each Small Area - uses a simple feature
smallAreasSF$impCount <- exact_extract(imperviousRaster, smallAreasSF, 'count')

#calculate the area (m2) in each small area of the raster data
smallAreasSF$impArea <- smallAreasSF$impCount*100

#calculate the area of impervious coverage in each small area
smallAreasSF$impCoverage <- (exact_extract(imperviousRaster, smallAreasSF, 'sum')/100)*cellArea

#calculate the percentage of each small area that is impervious
smallAreasSF$impPercent <- (smallAreasSF$impCoverage/smallAreasSF$impArea)*100

head(smallAreasSF)

Warning message in .local(x, y, ...):
“Polygons transformed to raster CRS (EPSG:NA)”
Cannot preload entire working area of 1395265641 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.



  |======================================================================| 100%


Warning message in .local(x, y, ...):
“Polygons transformed to raster CRS (EPSG:NA)”
Cannot preload entire working area of 1395265641 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.



  |======================================================================| 100%


,SA_GUID__1,COUNTY_ENG,geometry,impCount,impArea,impCoverage,impPercent
,<chr>,<chr>,<MULTIPOLYGON [m]>,<dbl>,<dbl>,<dbl>,<dbl>
1,4c07d11e-0a4f-851d-e053-ca3ca8c0ca7f,CARLOW,MULTIPOLYGON (((673146.5 67...,2049.913,204991.3,75945.95,37.04838
2,4c07d11e-0a3a-851d-e053-ca3ca8c0ca7f,CARLOW,MULTIPOLYGON (((673956.3 67...,2882.904,288290.4,57333.25,19.88733
3,4c07d11e-0a4e-851d-e053-ca3ca8c0ca7f,CARLOW,MULTIPOLYGON (((673720.6 67...,2084.744,208474.4,69274.20,33.22912
4,4c07d11e-0a25-851d-e053-ca3ca8c0ca7f,CARLOW,MULTIPOLYGON (((671923.3 67...,3488.803,348880.3,109519.96,31.39184
5,4c07d11e-0a57-851d-e053-ca3ca8c0ca7f,CARLOW,MULTIPOLYGON (((673711.8 67...,4294.469,429446.9,209251.55,48.72582
6,4c07d11e-2df2-851d-e053-ca3ca8c0ca7f,CARLOW,MULTIPOLYGON (((673488.5 67...,593.127,59312.7,18759.59,31.62829


### Z Scores

In [7]:
#scale the percentages to z scores
smallAreasSF$impervious_pct_Z <- scale(smallAreasSF$impPercent)
head(smallAreasSF)

,SA_GUID__1,COUNTY_ENG,geometry,impCount,impArea,impCoverage,impPercent,impervious_pct_Z
,<chr>,<chr>,<MULTIPOLYGON [m]>,<dbl>,<dbl>,<dbl>,<dbl>,"<dbl[,1]>"
1,4c07d11e-0a4f-851d-e053-ca3ca8c0ca7f,CARLOW,MULTIPOLYGON (((673146.5 67...,2049.913,204991.3,75945.95,37.04838,0.4175913
2,4c07d11e-0a3a-851d-e053-ca3ca8c0ca7f,CARLOW,MULTIPOLYGON (((673956.3 67...,2882.904,288290.4,57333.25,19.88733,-0.2955154
3,4c07d11e-0a4e-851d-e053-ca3ca8c0ca7f,CARLOW,MULTIPOLYGON (((673720.6 67...,2084.744,208474.4,69274.20,33.22912,0.2588866
4,4c07d11e-0a25-851d-e053-ca3ca8c0ca7f,CARLOW,MULTIPOLYGON (((671923.3 67...,3488.803,348880.3,109519.96,31.39184,0.1825408
5,4c07d11e-0a57-851d-e053-ca3ca8c0ca7f,CARLOW,MULTIPOLYGON (((673711.8 67...,4294.469,429446.9,209251.55,48.72582,0.9028331
6,4c07d11e-2df2-851d-e053-ca3ca8c0ca7f,CARLOW,MULTIPOLYGON (((673488.5 67...,593.127,59312.7,18759.59,31.62829,0.1923660


# Export

In [8]:
#EXPORT
#build export path
exportPath <- paste(exportDirectory, 'imperviousSurface', sep = '/')
exportPath <- paste(exportPath, '.csv', sep = '')

smallAreasBufferExport <- as.data.frame(smallAreasSF) %>% select(1,7,8)

names(smallAreasBufferExport) <- gsub("_pct_Z","",names(smallAreasBufferExport))

write.csv(smallAreasBufferExport, exportPath, row.names = FALSE)